## Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install pytorch_pretrained_vit
!pip3 install timm

import torch
from pathlib import Path
import os
from skimage import io
from PIL import Image
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import PIL
from pytorch_pretrained_vit import ViT
import timm
from torchvision.datasets.utils import download_url
import json
from torchvision.utils import make_grid, save_image

In [ ]:
#@title Directory to store input images.
Img_dir = "/content/drive/MyDrive/image_data/"

In [ ]:
#@title Normarization of image
center_crop = 224
preprocess = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

In [ ]:
#@title Calculation of probability of ImageNet 1k classes.
def calculate_probability(dir, preprocess, model):
  root_dir = dir
  model.eval()

  # download of labels of class
  if not Path("/content/data/imagenet_class_index.json").exists():
    download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

  with open("/content/data/imagenet_class_index.json") as f:
    data = json.load(f)
    class_names = [x["en"] for x in data] #English
    classes_japanese = [x["ja"] for x in data] #Japanese

  Angora_index = class_names.index('Angora')
  hare_index = class_names.index('hare')
  wood_rabbit_index = class_names.index('wood_rabbit')

  data_dir = Path(root_dir) #go into root directory
  data_files = data_dir.glob('*.png') #Images by .png name

  for image in data_files: #every image in the directory
      #chages to 3channels
      input = Image.open(image).convert('RGB')
      input_tensor = preprocess(input)
      input_batch = input_tensor.unsqueeze(0)
      input_batch = input_batch.to('cuda')

      with torch.no_grad():
        output_probs = model(input_batch)
        softmax_output = torch.nn.functional.softmax(output_probs[0], dim=0)
        batch_probs, batch_indices = softmax_output.sort(dim=0,descending=True)
        #probability for calculation of rabbit class (not sorted)
        batch_probs_rabbits = softmax_output

        print(image)

        #print up to the tenth probabile class
        for k in range(10):
          prob = batch_probs[k]
          prob_numpy = prob.cpu().numpy()
          indices = batch_indices[k]
          indices_numpy = indices.cpu().numpy()
          class_name = class_names[indices_numpy]
          class_name_japanese = classes_japanese[indices_numpy]
          print(k+1, indices_numpy, class_name, class_name_japanese, prob_numpy)

        print("")
        #calculation of probability of rabbit class
        Angora_prob = batch_probs_rabbits[Angora_index].cpu().numpy()
        hare_prob = batch_probs_rabbits[hare_index].cpu().numpy()
        wood_rabbit_prob = batch_probs_rabbits[wood_rabbit_index].cpu().numpy()

        print("Angora", Angora_index, classes_japanese[Angora_index], Angora_prob)
        print("hare", hare_index, classes_japanese[hare_index], hare_prob)
        print("wood_rabbit", wood_rabbit_index, classes_japanese[wood_rabbit_index], wood_rabbit_prob)
        #rabbit class is the sum of the three hare classes.
        print("all_rabbit", Angora_prob + hare_prob + wood_rabbit_prob)
        print("")

In [ ]:
#@title Number of parameters
def num_params(model):
  return sum(p.numel() for p in model.parameters())

## Models to calculate probabilities.

In [ ]:
#@title Resnet50
#Resnet50
model = timm.create_model('resnet50', pretrained=True)
model = model.to('cuda')

#calculate the number of parameters
print("NUM PARAMS")
print(num_params(model))

calculate_probability(Img_dir, preprocess, model)

NUM PARAMS
25557032
/content/drive/MyDrive/image_data/moon_binary_th90_resize_rot100.png
1 574 golf_ball ゴルフボール 0.12674448
2 952 fig イチジク 0.044774562
3 254 pug パグ 0.03645973
4 954 banana バナナ 0.034713257
5 251 dalmatian ダルメシアン 0.03250973
6 949 strawberry イチゴ 0.02944645
7 773 saltshaker 塩の入れ物 0.018724533
8 948 Granny_Smith リンゴ 0.018256517
9 276 hyena ハイエナ 0.017183738
10 78 tick ダニ 0.01684179

Angora 332 アンゴラ 0.0001191535
hare 331 野ウサギ 0.00032122972
wood_rabbit 330 木のウサギ 0.0006840913
all_rabbit 0.0011244745

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_rot100.png
1 78 tick ダニ 0.21164003
2 126 isopod 等脚類 0.03188757
3 714 pick 選ぶ 0.02759906
4 111 nematode 線虫 0.02709521
5 712 Petri_dish ペトリ皿 0.020694973
6 216 clumber クランバー 0.020504473
7 190 Sealyham_terrier シーリーハムテリア 0.017268572
8 228 komondor コモンドール 0.016009413
9 251 dalmatian ダルメシアン 0.013990806
10 641 maraca マラカス 0.011327896

Angora 332 アンゴラ 0.0012779043
hare 331 野ウサギ 0.00083161285
wood_rabbit 330 木のウサギ 0.00106496

In [ ]:
#@title VIT Large 16
# pre-trained model from https://github.com/rwightman/pytorch-image-models
model = timm.create_model('vit_large_patch16_224', pretrained=True)
model = model.to('cuda')

#calculate the number of parameters
print("NUM PARAMS")
print(num_params(model))

calculate_probability(Img_dir, preprocess, model)

NUM PARAMS
304326632
/content/drive/MyDrive/image_data/moon_binary_th90_resize_rot100.png
1 980 volcano 火山 0.07739
2 971 bubble バブル 0.06939507
3 727 planetarium プラネタリウム 0.057463367
4 712 Petri_dish ペトリ皿 0.026074303
5 635 magnetic_compass 磁気コンパス 0.023759598
6 574 golf_ball ゴルフボール 0.020848125
7 506 coil コイル 0.019256594
8 417 balloon バルーン 0.015095153
9 599 honeycomb ハニカム 0.011795016
10 807 solar_dish 太陽の皿 0.011723848

Angora 332 アンゴラ 0.00032554436
hare 331 野ウサギ 5.207989e-05
wood_rabbit 330 木のウサギ 0.000115222596
all_rabbit 0.0004928468

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_rot100.png
1 712 Petri_dish ペトリ皿 0.1470877
2 111 nematode 線虫 0.0791042
3 577 gong ゴング 0.046105064
4 551 face_powder フェースパウダー 0.03684343
5 635 magnetic_compass 磁気コンパス 0.034404695
6 574 golf_ball ゴルフボール 0.034269925
7 633 loupe ルーペ 0.032531008
8 727 planetarium プラネタリウム 0.031663366
9 772 safety_pin 安全ピン 0.019455204
10 677 nail ネイル 0.01591731

Angora 332 アンゴラ 0.00019134043
hare 331 野ウサギ 6.7451

In [ ]:
#@title Bit_Resnet_152x2 trained with imagenet 21k and finetuned with 1k
model = timm.create_model('resnetv2_152x2_bitm', pretrained=True)
model = model.to('cuda')

#calculate the number of parameters
print("NUM PARAMS")
print(num_params(model))

calculate_probability(Img_dir, preprocess, model)

NUM PARAMS
236335208
/content/drive/MyDrive/image_data/moon_binary_th90_resize_rot100.png
1 971 bubble バブル 0.34315452
2 641 maraca マラカス 0.08540391
3 107 jellyfish クラゲ 0.05171262
4 574 golf_ball ゴルフボール 0.030401532
5 438 beaker ビーカー 0.02772934
6 712 Petri_dish ペトリ皿 0.024694227
7 722 ping-pong_ball ピンポン球 0.021624442
8 746 puck パック 0.020753574
9 980 volcano 火山 0.019987538
10 727 planetarium プラネタリウム 0.016392834

Angora 332 アンゴラ 1.8936284e-05
hare 331 野ウサギ 0.0002595341
wood_rabbit 330 木のウサギ 0.00011603342
all_rabbit 0.00039450382

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_rot100.png
1 971 bubble バブル 0.2550384
2 712 Petri_dish ペトリ皿 0.24019758
3 111 nematode 線虫 0.086487874
4 107 jellyfish クラゲ 0.06268442
5 438 beaker ビーカー 0.030992249
6 980 volcano 火山 0.022301784
7 883 vase 花瓶 0.018169735
8 78 tick ダニ 0.009558619
9 952 fig イチジク 0.008829485
10 251 dalmatian ダルメシアン 0.0059296764

Angora 332 アンゴラ 8.626413e-05
hare 331 野ウサギ 0.00050282746
wood_rabbit 330 木のウサギ 0.0005770069


In [ ]:
#@title SWSL resnext101_32x8d
model = timm.create_model('swsl_resnext101_32x8d', pretrained=True)
model = model.to('cuda')

#calculate the number of parameters
print("NUM PARAMS")
print(num_params(model))

calculate_probability(Img_dir, preprocess, model)

NUM PARAMS
88791336
/content/drive/MyDrive/image_data/moon_binary_th90_resize_rot100.png
1 712 Petri_dish ペトリ皿 0.22792044
2 971 bubble バブル 0.21555081
3 567 frying_pan フライパン 0.061918277
4 960 chocolate_sauce チョコレートソース 0.02418005
5 577 gong ゴング 0.022165436
6 417 balloon バルーン 0.018123817
7 828 strainer ストレーナー 0.013093119
8 845 syringe 注射器 0.012423129
9 805 soccer_ball サッカーボール 0.011935715
10 696 paintbrush 絵筆 0.010005887

Angora 332 アンゴラ 1.2227655e-05
hare 331 野ウサギ 0.00012648932
wood_rabbit 330 木のウサギ 2.545069e-05
all_rabbit 0.00016416766

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_rot100.png
1 574 golf_ball ゴルフボール 0.20349939
2 577 gong ゴング 0.10910763
3 712 Petri_dish ペトリ皿 0.099647924
4 111 nematode 線虫 0.085016266
5 956 custard_apple カスタードアップル 0.04898824
6 409 analog_clock アナログ時計 0.0271307
7 78 tick ダニ 0.021030733
8 116 chiton キトン 0.016722936
9 635 magnetic_compass 磁気コンパス 0.016503025
10 522 croquet_ball クロケットボール 0.012235912

Angora 332 アンゴラ 0.0001924375
hare 331 

In [ ]:
#@title Convnext 21k finetuned 1k
model = timm.create_model('convnext_large_in22ft1k', pretrained=True)
model = model.to('cuda')

#calculate the number of parameters
print("NUM PARAMS")
print(num_params(model))

calculate_probability(Img_dir, preprocess, model)

NUM PARAMS
197767336
/content/drive/MyDrive/image_data/moon_binary_th90_resize_rot100.png
1 956 custard_apple カスタードアップル 0.4829189
2 952 fig イチジク 0.14480226
3 574 golf_ball ゴルフボール 0.027258636
4 954 banana バナナ 0.023782162
5 988 acorn ドングリ 0.012533263
6 957 pomegranate ザクロ 0.009571756
7 949 strawberry イチゴ 0.009320259
8 948 Granny_Smith リンゴ 0.008251645
9 522 croquet_ball クロケットボール 0.006769886
10 852 tennis_ball テニスボール 0.0066396915

Angora 332 アンゴラ 0.00013345636
hare 331 野ウサギ 5.879165e-05
wood_rabbit 330 木のウサギ 6.223461e-05
all_rabbit 0.0002544826

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_rot100.png
1 574 golf_ball ゴルフボール 0.11553067
2 111 nematode 線虫 0.101657785
3 712 Petri_dish ペトリ皿 0.076447494
4 641 maraca マラカス 0.024200762
5 684 ocarina オカリナ 0.020366758
6 78 tick ダニ 0.015712712
7 883 vase 花瓶 0.015452221
8 305 dung_beetle フンコロガシ 0.010692737
9 722 ping-pong_ball ピンポン球 0.0093908
10 805 soccer_ball サッカーボール 0.008328652

Angora 332 アンゴラ 0.0019050875
hare 331 野ウサギ 0.0

In [ ]:
#@title Noisy Student (Efficient net b0  image size= 224 )
model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
model = model.to('cuda')

#calculate the number of parameters
print("NUM PARAMS")
print(num_params(model))

calculate_probability(Img_dir, preprocess, model)

NUM PARAMS
5288548
/content/drive/MyDrive/image_data/moon_binary_th90_resize_rot100.png
1 310 ant 蟻 0.07333849
2 78 tick ダニ 0.06765252
3 361 skunk スカンク 0.0330708
4 644 matchstick マッチ棒 0.026495675
5 937 broccoli ブロッコリー 0.020944754
6 328 sea_urchin うに 0.019176273
7 712 Petri_dish ペトリ皿 0.014368609
8 980 volcano 火山 0.0141044725
9 111 nematode 線虫 0.013235685
10 506 coil コイル 0.011270197

Angora 332 アンゴラ 0.000629004
hare 331 野ウサギ 0.0007027571
wood_rabbit 330 木のウサギ 0.00038102063
all_rabbit 0.0017127816

/content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_rot100.png
1 111 nematode 線虫 0.114630215
2 78 tick ダニ 0.032947015
3 956 custard_apple カスタードアップル 0.015699094
4 116 chiton キトン 0.014419384
5 712 Petri_dish ペトリ皿 0.012728925
6 644 matchstick マッチ棒 0.011469424
7 577 gong ゴング 0.01137762
8 183 Kerry_blue_terrier ケリーブルーテリア 0.011157371
9 117 chambered_nautilus オウムガイ 0.010340679
10 362 badger 狸 0.009876339

Angora 332 アンゴラ 0.0007272854
hare 331 野ウサギ 0.00137476
wood_rabbit 330 木のウサギ 0.